In [ ]:
#ANN 10

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

# Define the label map
label_map = {1: 'person', 2: 'car', 3: 'dog'}  # Replace with your label map

path = r"C:\Users\Avina\ANN practical\test.png"

# Read the image using OpenCV
image = cv2.imread(path)
if image is None:
    print("Error: Image not loaded. Check file path or file existence.")
else:
    # Convert image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Prepare the image for the model input
    image_resized = cv2.resize(image_rgb, (320, 320))  # Resize to the input size of the model
    image_tensor = np.expand_dims(image_resized, axis=0)  # Add batch dimension
    image_tensor = tf.convert_to_tensor(image_tensor, dtype=tf.uint8)

    # Perform object detection
    detections = model(image_tensor)
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(int)

    # Set the threshold for detections
    threshold = 0.5
    filtered_boxes = boxes[scores >= threshold]
    filtered_scores = scores[scores >= threshold]
    filtered_classes = classes[scores >= threshold]

    # Prepare data for printing predictions
    prediction_data = []

    for box, cls, score in zip(filtered_boxes, filtered_classes, filtered_scores):
        ymin, xmin, ymax, xmax = box
        ymin *= image.shape[0]
        xmin *= image.shape[1]
        ymax *= image.shape[0]
        xmax *= image.shape[1]

        label = label_map.get(cls, 'Unknown')
        prediction_data.append([xmin, ymin, xmax, ymax, score, cls, label])

        # Draw bounding box and label on the image
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        cv2.putText(image, label, (int(xmin), int(ymin) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Convert back for displaying
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Display the image with bounding boxes
    cv2.imshow('Detected Objects', image_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f"Boxes detected: {len(filtered_boxes)}\n")

    # ----------- Print predictions neatly in table ----------- #
    columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class_id', 'class_name']
    predictions_df = pd.DataFrame(prediction_data, columns=columns)
    predictions_df.index.name = 'Index'

    # Print predictions
    print(predictions_df)

# ------- Evaluation functions -------- #

def compute_iou(box1, box2):
    """Compute IoU between two bounding boxes."""
    x1 = max(box1[1], box2[1])
    y1 = max(box1[0], box2[0])
    x2 = min(box1[3], box2[3])
    y2 = min(box1[2], box2[2])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = (box1[3] - box1[1]) * (box1[2] - box1[0])
    box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])

    union = box1_area + box2_area - intersection

    if union == 0:
        return 0

    return intersection / union

def evaluate_object_detection(pred_boxes, pred_scores, pred_classes, gt_boxes, iou_threshold=0.5):
    """Evaluate object detection performance."""
    matched_gt = set()
    tp, fp = [], []

    for i, pred in enumerate(pred_boxes):
        best_iou = 0
        best_gt_idx = -1
        for j, gt in enumerate(gt_boxes):
            if gt[4] != pred_classes[i]:
                continue
            iou = compute_iou(pred, gt[:4])
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = j

        if best_iou >= iou_threshold and best_gt_idx not in matched_gt:
            tp.append(1)
            fp.append(0)
            matched_gt.add(best_gt_idx)
        else:
            tp.append(0)
            fp.append(1)

    tp_cumsum = np.cumsum(tp)
    fp_cumsum = np.cumsum(fp)
    recall = tp_cumsum / len(gt_boxes)
    precision = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)

    return precision, recall

def interpolate_11_point(precision, recall):
    """Compute 11-point interpolated Average Precision."""
    recall_levels = np.linspace(0, 1, 11)
    interpolated = []

    for r in recall_levels:
        precisions_above_r = precision[recall >= r]
        max_p = max(precisions_above_r) if precisions_above_r.size > 0 else 0
        interpolated.append(max_p)

    ap = np.mean(interpolated)
    return ap

# Example ground truth boxes (add your actual ground truth boxes)
gt_boxes = [
    [0.3, 0.4, 0.6, 0.7, 2],   # car
    [0.2, 0.1, 0.5, 0.4, 1],   # person
    # Add more ground truth boxes here
]

# Evaluate the object detection
precision, recall = evaluate_object_detection(filtered_boxes, filtered_scores, filtered_classes, gt_boxes)

print("\nPrecision:", precision)
print("Recall:", recall)

# Compute Average Precision
ap = interpolate_11_point(precision, recall)
print("\n11-Point Interpolated AP:", ap)
